# Sistema completo

In [14]:
import cv2
import mediapipe as mp
import time

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

LEFT_EYE_FULL = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_FULL = [362, 385, 387, 263, 373, 380]

def eye_aspect_ratio(landmarks, eye_indices, image_shape):
    def dist(p1, p2):
        x1, y1 = int(p1.x * image_shape[1]), int(p1.y * image_shape[0])
        x2, y2 = int(p2.x * image_shape[1]), int(p2.y * image_shape[0])
        return ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
    A = dist(landmarks[eye_indices[1]], landmarks[eye_indices[5]])
    B = dist(landmarks[eye_indices[2]], landmarks[eye_indices[4]])
    D = dist(landmarks[eye_indices[0]], landmarks[eye_indices[3]])
    return (A + B) / (2.0 * D)

def is_looking_forward(landmarks):
    def centered(pupil, outer, inner):
        d1 = abs(pupil - outer)
        d2 = abs(inner - pupil)
        ratio = d1 / (d1 + d2 + 1e-6)
        return 0.425 < ratio < 0.575
    left = centered(landmarks[468].x, landmarks[33].x, landmarks[133].x)
    right = centered(landmarks[473].x, landmarks[362].x, landmarks[263].x)
    return left and right

def realizar_reto_de_vida():
    cap = cv2.VideoCapture(0)
    frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter("verificacion_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 20, (frame_width, frame_height))
    face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

    # Estado
    blink_counter, blink_ready = 0, True
    blink_start_time, blink_completed = None, False
    nod_count, nod_stage, nod_start_y, nod_start_time, nod_completed = 0, "neutral", None, None, False
    shake_count, shake_stage, shake_start_x, shake_start_time, shake_completed = 0, "neutral", None, None, False
    stable_frame, stable_counter = None, 0
    gaze_start_time, selfie_taken = None, False
    fase_selfie = False

    print("🟡 Iniciando reto de vida...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        raw_frame = frame.copy()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)

        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

            ear_left = eye_aspect_ratio(face_landmarks.landmark, LEFT_EYE_FULL, frame.shape)
            ear_right = eye_aspect_ratio(face_landmarks.landmark, RIGHT_EYE_FULL, frame.shape)
            ear_avg = (ear_left + ear_right) / 2.0

            if not fase_selfie:
                # Parpadeo
                if not blink_completed:
                    if ear_avg < 0.26 and blink_ready:
                        if blink_counter == 0:
                            blink_start_time = time.time()
                        blink_counter += 1
                        blink_ready = False
                        print(f"✅ Parpadeo #{blink_counter}")
                    elif ear_avg >= 0.30:
                        blink_ready = True
                    if blink_counter > 0 and (time.time() - blink_start_time > 3):
                        print("⏱️ Parpadeos: tiempo excedido")
                        blink_counter, blink_start_time = 0, None
                    if blink_counter >= 3:
                        blink_completed = True
                        print("✅ Parpadeos completados")

                # Asentir ("sí")
                nose_y = face_landmarks.landmark[1].y
                if nod_start_y is None:
                    nod_start_y = nose_y
                    nod_start_time = time.time()
                delta_y = nose_y - nod_start_y
                if not nod_completed:
                    if nod_stage == "neutral" and delta_y > 0.03:
                        nod_stage = "down"
                    elif nod_stage == "down" and delta_y < -0.03:
                        nod_stage = "up"
                        nod_count += 1
                        print(f"✅ Asentimiento #{nod_count}")
                        nod_stage = "neutral"
                    if time.time() - nod_start_time > 5 and nod_count < 2:
                        print("⏱️ Asentir: tiempo excedido")
                        nod_count, nod_start_time = 0, time.time()
                    if nod_count >= 2:
                        nod_completed = True
                        print("✅ Asentir completado")

                # Negar ("no")
                nose_x = face_landmarks.landmark[1].x
                if shake_start_x is None:
                    shake_start_x = nose_x
                    shake_start_time = time.time()
                delta_x = nose_x - shake_start_x
                if not shake_completed:
                    if shake_stage == "neutral" and delta_x > 0.03:
                        shake_stage = "right"
                    elif shake_stage == "right" and delta_x < -0.03:
                        shake_stage = "left"
                        shake_count += 1
                        print(f"✅ Negación #{shake_count}")
                        shake_stage = "neutral"
                    if time.time() - shake_start_time > 5 and shake_count < 2:
                        print("⏱️ Negar: tiempo excedido")
                        shake_count, shake_start_time = 0, time.time()
                    if shake_count >= 2:
                        shake_completed = True
                        print("✅ Negar completado")

                # Mostrar progreso
                cv2.putText(frame, f"Parpadeos: {blink_counter}/3", (30, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.putText(frame, f"Asentir: {nod_count}/2", (30, 70),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                cv2.putText(frame, f"Negar: {shake_count}/2", (30, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)

                if blink_completed and nod_completed and shake_completed:
                    print("📸 Reto de vida completado. Prepara tu selfie...")
                    fase_selfie = True
                    start_selfie_time = time.time()
                    continue

            if fase_selfie and not selfie_taken:
                nose_x = face_landmarks.landmark[1].x
                nose_y = face_landmarks.landmark[1].y
                centered = 0.4 < nose_x < 0.6 and 0.4 < nose_y < 0.6
                eyes_open = ear_left > 0.26 and ear_right > 0.26

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (21, 21), 0)
                if stable_frame is None:
                    stable_frame = gray
                    continue
                diff = cv2.absdiff(stable_frame, gray)
                _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
                movement = cv2.countNonZero(thresh)
                stable = movement < 5000
                stable_counter = stable_counter + 1 if stable else 0

                looking_forward = is_looking_forward(face_landmarks.landmark)
                if looking_forward:
                    if gaze_start_time is None:
                        gaze_start_time = time.time()
                    gaze_duration = time.time() - gaze_start_time
                else:
                    gaze_start_time = None
                    gaze_duration = 0

                cv2.putText(frame, f"Cara centrada: {'✅' if centered else '❌'}", (30, 140),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
                cv2.putText(frame, f"Ojos abiertos: {'✅' if eyes_open else '❌'}", (30, 170),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
                cv2.putText(frame, f"Estable: {'✅' if stable_counter >= 5 else '❌'}", (30, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
                cv2.putText(frame, f"Mirando: {'✅' if gaze_duration >= 3 else f'{gaze_duration:.1f}s'}", (30, 230),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)

                if centered and eyes_open and stable_counter >= 5 and gaze_duration >= 3:
                    cv2.imwrite("selfie.jpg", raw_frame)
                    print("📷 Selfie capturada como 'selfie.jpg'")
                    selfie_taken = True
                    break
                stable_frame = gray

        cv2.imshow("Verificación Facial", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("🎬 Reto de vida completado correctamente.")
    if selfie_taken == True:
        return True
    else:
        return False


In [3]:
import torch
print("¿GPU disponible?:", torch.cuda.is_available())
print("GPU actual:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Ninguna")


¿GPU disponible?: True
GPU actual: NVIDIA GeForce RTX 4070 Ti


In [18]:
def verificar_ine_con_curp():
    import cv2
    import time
    import re
    import imutils
    from selenium import webdriver
    from selenium.common.exceptions import WebDriverException, TimeoutException
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from paddleocr import PaddleOCR

    def coincide_nombre_web_con_ocr(nombre_web, ap1, ap2, lineas_ocr):
        partes = [nombre_web, ap1, ap2]
        for parte in partes:
            if not any(parte.upper() in linea for linea in lineas_ocr):
                return False
        return True

    ocr = PaddleOCR(use_angle_cls=True, lang='es')
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    while True:
        print("🪪 Muestra tu INE al centro de la cámara, bien enfocada y estable...")

        cap = cv2.VideoCapture(0)
        stable_frame = None
        ine_capturada = False
        stable_counter = 0
        required_frames = 150

        while cap.isOpened():
            ret, raw_frame = cap.read()
            if not ret:
                break

            frame = raw_frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edged = cv2.Canny(blur, 75, 200)
            cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

            found_card = False
            frame_area = frame.shape[0] * frame.shape[1]

            for c in cnts:
                approx = cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)
                area = cv2.contourArea(c)
                if len(approx) == 4 and area > 0.20 * frame_area:
                    cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
                    stable_counter += 1
                    found_card = True
                    break

            if not found_card:
                stable_counter = 0

            porcentaje = int((stable_counter / required_frames) * 100)
            cv2.putText(frame, f"INE detectada: {porcentaje}%", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, "Mantén la INE visible y grande durante 5 segundos",
                        (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

            cv2.imshow("Detección de INE", frame)

            if stable_counter >= required_frames:
                cv2.imwrite("ine.jpg", raw_frame)
                print("📸 INE capturada como 'ine.jpg'")
                ine_capturada = True
                break

            if cv2.waitKey(1) & 0xFF == 27:
                cap.release()
                cv2.destroyAllWindows()
                return False

        cap.release()
        cv2.destroyAllWindows()

        if not ine_capturada:
            continue

        print("🔍 Procesando INE con OCR...")
        results = ocr.ocr("ine.jpg", cls=True)
        lineas_ocr_ine = [line[1][0].strip().upper() for line in results[0]]

        curp_match = next((re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", l)
                           for l in lineas_ocr_ine if re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", l)), None)
        curp = curp_match.group() if curp_match else "No detectado"

        fecha_match = next((re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", l)
                            for l in lineas_ocr_ine if re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", l)), None)
        fecha = fecha_match.group() if fecha_match else "No detectada"

        print("📌 CURP detectado:", curp)
        print("📌 Fecha de nacimiento:", fecha)

        print("🌐 Consultando datos oficiales del CURP...")
        try:
            driver = webdriver.Chrome(options=chrome_options)
            driver.get("https://www.gob.mx/curp/")

            input_curp = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "curpinput"))
            )
            input_curp.send_keys(curp)
            boton_buscar = driver.find_element(By.ID, "searchButton")
            boton_buscar.click()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//td[@style='text-transform: uppercase;']"))
            )

            celdas = driver.find_elements(By.XPATH, "//td[@style='text-transform: uppercase;']")
            if len(celdas) >= 7:
                nombre_web = celdas[1].text.strip()
                apellido1_web = celdas[2].text.strip()
                apellido2_web = celdas[3].text.strip()
                fecha_web = celdas[5].text.strip()

                print("📄 Datos oficiales:", apellido1_web, apellido2_web, nombre_web, fecha_web)
                nombre_ok = coincide_nombre_web_con_ocr(nombre_web, apellido1_web, apellido2_web, lineas_ocr_ine)
                fecha_ok = (fecha.strip() == fecha_web.strip())

                if nombre_ok and fecha_ok:
                    print("✅ Identidad confirmada con datos oficiales.")
                    driver.quit()
                    return True
                else:
                    print("❌ Los datos de la INE no coinciden con el CURP.")
            else:
                print("❌ No se extrajeron suficientes datos del sitio.")
            driver.quit()
            time.sleep(2)

        except (WebDriverException, TimeoutException) as e:
            print(f"⚠️ Error en Selenium: {e}")
            print("🔁 Ocurrió un error al consultar el CURP. Volveremos a escanear la INE...")
            try:
                driver.quit()
            except:
                pass
            time.sleep(2)
            continue  # vuelve al loop a escanear de nuevo



In [20]:
def comparar_rostros_ine_selfie():
    import mediapipe as mp
    import face_recognition
    import cv2
    import os

    print("🧠 Detectando rostro en la INE...")

    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.6)

    image = cv2.imread("ine.jpg")
    if image is None:
        print("❌ No se encontró 'ine.jpg'.")
        return False

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)

    if results.detections:
        h, w, _ = image.shape
        max_area = 0
        best_face = None

        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            x, y = int(bbox.xmin * w), int(bbox.ymin * h)
            w_box, h_box = int(bbox.width * w), int(bbox.height * h)
            area = w_box * h_box

            if area > max_area:
                max_area = area
                best_face = (x, y, w_box, h_box)

        if best_face:
            x, y, w_box, h_box = best_face
            x, y = max(0, x), max(0, y)
            cropped_face = image[y:y + h_box, x:x + w_box]
            cv2.imwrite("ine_face.jpg", cropped_face)
            print("✅ Rostro recortado y guardado como 'ine_face.jpg'")
    else:
        print("❌ No se detectó rostro en la INE.")
        return False

    # Comparación facial
    print("🧪 Comparando rostro de INE con selfie...")

    if not os.path.exists("selfie.jpg") or not os.path.exists("ine_face.jpg"):
        print("❌ Faltan imágenes para la comparación facial.")
        return False

    img1 = face_recognition.load_image_file("selfie.jpg")
    img2 = face_recognition.load_image_file("ine_face.jpg")

    enc1 = face_recognition.face_encodings(img1)
    enc2 = face_recognition.face_encodings(img2)

    if enc1 and enc2:
        result = face_recognition.compare_faces([enc1[0]], enc2[0])
        distance = face_recognition.face_distance([enc1[0]], enc2[0])[0]

        if result[0]:
            print(f"✅ Rostros coinciden (distancia: {distance:.4f})")
            return True
        else:
            print(f"❌ Rostros NO coinciden (distancia: {distance:.4f})")
            return False
    else:
        print("❌ No se pudieron codificar ambos rostros correctamente.")
        return False


In [22]:
def predecir_deepfake(video_path, model_path="mediapipe_model.pth"):
    import torch
    import torch.nn as nn
    import cv2
    import numpy as np
    from torchvision import transforms, models
    import mediapipe as mp
    import os

    class DeepfakeDetector(nn.Module):
        def __init__(self):
            super().__init__()
            self.cnn = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
            self.cnn.classifier = nn.Identity()
            self.embedding_dim = 1280
            self.sequence_length = 16

            self.lstm = nn.LSTM(input_size=1285, hidden_size=128, num_layers=1,
                                batch_first=True, bidirectional=True)
            self.fc = nn.Sequential(
                nn.Linear(256, 64),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(64, 1),
                nn.Sigmoid()
            )

        def forward(self, x_imgs, x_lmks):
            B, T, C, H, W = x_imgs.shape
            x_imgs = x_imgs.view(B * T, C, H, W)
            features = self.cnn(x_imgs)
            features = features.view(B, T, -1)
            combined = torch.cat([features, x_lmks], dim=2)
            out, _ = self.lstm(combined)
            out = out[:, -1, :]
            return self.fc(out).squeeze(1)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    sequence_length = 16
    candidate_frames = 25
    image_size = (256, 256)
    transform = transforms.Compose([transforms.ToTensor()])
    mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                                   refine_landmarks=True, min_detection_confidence=0.5)

    def extract_landmark_vector(landmarks, frame_shape):
        h, w, _ = frame_shape
        def norm(x): return x / w
        def norm_y(y): return y / h
        left_eye = landmarks.landmark[33]
        right_eye = landmarks.landmark[263]
        nose = landmarks.landmark[1]
        mouth_left = landmarks.landmark[61]
        mouth_right = landmarks.landmark[291]
        return np.array([
            norm(left_eye.x), norm(right_eye.x),
            norm_y(nose.y),
            norm_y(mouth_left.y),
            norm_y(mouth_right.y)
        ], dtype=np.float32)

    def crop_face_from_landmarks(landmarks, frame):
        h, w, _ = frame.shape
        x_coords = [lm.x for lm in landmarks.landmark]
        y_coords = [lm.y for lm in landmarks.landmark]
        min_x, max_x = int(min(x_coords) * w), int(max(x_coords) * w)
        min_y, max_y = int(min(y_coords) * h), int(max(y_coords) * h)
        margin_x = int((max_x - min_x) * 0.2)
        margin_y = int((max_y - min_y) * 0.2)
        x1 = max(min_x - margin_x, 0)
        y1 = max(min_y - margin_y, 0)
        x2 = min(max_x + margin_x, w)
        y2 = min(max_y + margin_y, h)
        face_crop = frame[y1:y2, x1:x2]
        return cv2.resize(face_crop, image_size)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, candidate_frames).astype(int)

    images, landmarks_list = [], []
    evidencia_guardada = False

    for idx in frame_indices:
        if len(images) >= sequence_length:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mp_face_mesh.process(rgb)
        if results.multi_face_landmarks:
            try:
                lmks = results.multi_face_landmarks[0]
                cropped = crop_face_from_landmarks(lmks, frame)
                lmk_vector = extract_landmark_vector(lmks, frame.shape)

                if not evidencia_guardada:
                    cv2.imwrite("evidencia.jpg", cropped)
                    evidencia_guardada = True

                images.append(transform(cropped))
                landmarks_list.append(torch.tensor(lmk_vector, dtype=torch.float32))
            except:
                continue

    cap.release()
    mp_face_mesh.close()

    if len(images) < sequence_length:
        print(f"⚠️ Solo se obtuvieron {len(images)} frames válidos. No se puede hacer inferencia.")
        return None

    x_imgs = torch.stack(images[:sequence_length]).unsqueeze(0).to(device)
    x_lmks = torch.stack(landmarks_list[:sequence_length]).unsqueeze(0).to(device)

    model = DeepfakeDetector().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        output = model(x_imgs, x_lmks)
        prob = output.item()
        label = "FAKE" if prob > 0.5 else "REAL"
        print(f"\n🧪 Resultado: {label}  |  Probabilidad: {prob:.4f} | Evidencia: evidencia.jpg")
        return {"label": label, "prob": prob, "evidencia": "evidencia.jpg"}


In [26]:
if realizar_reto_de_vida() == True:
    if verificar_ine_con_curp() == True:
        if comparar_rostros_ine_selfie() == True:
            resultado = predecir_deepfake("verificacion_video.mp4", "mediapipe_model.pth")
            if resultado:
                print("Veredicto final: Es una persona", resultado["label"])
            else:
                print("❌ No se pudo analizar el video.")
        else:
            print("No se pudo comprobar que se tratara de una persona real")
    else:
        print("No se pudo comprobar que se tratara de una persona real")
else:
    print("No se pudo comprobar que se tratara de una persona real")

🟡 Iniciando reto de vida...
✅ Parpadeo #1
✅ Parpadeo #2
✅ Negación #1
✅ Parpadeo #3
✅ Parpadeos completados
⏱️ Asentir: tiempo excedido
⏱️ Negar: tiempo excedido
✅ Asentimiento #1
✅ Asentimiento #2
✅ Asentir completado
✅ Negación #1
⏱️ Negar: tiempo excedido
✅ Negación #1
✅ Negación #2
✅ Negar completado
📸 Reto de vida completado. Prepara tu selfie...
📷 Selfie capturada como 'selfie.jpg'
🎬 Reto de vida completado correctamente.
[2025/05/07 22:43:15] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast'